In [0]:
from pyspark import *
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains, struct, collect_list, row_number


# Transformaciones con Tipos de Datos Complejos: Arrays, Structs y Maps

En este notebook:
- Trabajaremos con columnas tipo array y struct a partir del dataset MovieLens.
- Aplicaremos funciones como `explode`, y crearemos structs de varias columnas.


In [0]:
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/ratings_full.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')

# Tabla Movies
movies_schema  = StructType(fields=[
    StructField("movieId",IntegerType(),True), 
    StructField("title",StringType(),True),
    StructField("genres",StringType(),True)
])

moviesDf = spark.read\
    .option("header", True)\
    .schema(movies_schema)\
    .csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/movies.csv")

moviesDf = moviesDf.withColumn("genresSplit", split(moviesDf["genres"],"\|"))\
                        .drop('genres').withColumnRenamed("genresSplit","genres")\
                            .withColumn(\
                                "year",\
                                regexp_extract(\
                                           moviesDf["title"],\
                                           "^.+\(([0-9]+)\)$",\
                                           1)\
                                .try_cast(IntegerType()))\
                            .withColumn(\
                            "title_temp",\
                            regexp_extract(\
                                           moviesDf["title"],\
                                           "^(.+?) \([0-9]+\)$",\
                                           1))\
                            .drop('title')\
                        .withColumnRenamed("title_temp","title")




In [0]:
# Agrupar calificaciones por usuario (array de movieIds)
user_movies = ratingsDf.groupBy("userId").agg(collect_list("movieId").alias("peliculas_calificadas"))
user_movies.show(5, truncate=False)



In [0]:
# Explode: expandir para tener una fila por usuario/película
user_movies_exploded = user_movies.withColumn("pelicula_id", explode(col("peliculas_calificadas")))
user_movies_exploded.show(5)



In [0]:
# Struct: juntar nombre y género de la película
movies_struct = moviesDf.withColumn("info", struct("title", "genres"))
movies_struct.select("movieId", "info").show(5, truncate=False)



In [0]:
# Crear columna tipo map (clave=movieId, valor=rating) para cada usuario
from pyspark.sql.functions import create_map, lit
# Ejemplo sencillo: para cada usuario, tomar sus primeras 3 calificaciones y armar el map
sample = ratingsDf.filter(col("userId") == 1).limit(3)
sample_map = sample.withColumn("movie_rating_map", create_map(col("movieId"), col("rating")))
sample_map.show()

In [0]:
# Tomemos los 3 primeros ratings de cada usuario para simplicidad
w = Window.partitionBy("userId").orderBy("date")
ratings_sample = ratingsDf.withColumn("rn", row_number().over(w)).filter(col("rn") <= 3)

# Para cada usuario, agrupamos las (movieId, rating) en una lista de pares
user_pairs = ratings_sample.groupBy("userId").agg(collect_list(struct("movieId", "rating")).alias("peliculas_ratings"))

# Convertimos la lista de structs a una columna Map
from pyspark.sql.functions import map_from_entries

user_map = user_pairs.withColumn("mapa_ratings", map_from_entries(col("peliculas_ratings")))

# Mostramos el resultado: ahora para cada usuario hay un Map movieId->rating
user_map.select("userId", "mapa_ratings").show(truncate=False)



In [0]:
# --- USO DEL MAP ---
user_map_con_valor = user_map.withColumn("rating_pelicula_1", col("mapa_ratings").getItem(lit(1)))
user_map_con_valor.select("userId", "mapa_ratings", "rating_pelicula_1").show()



In [0]:
# Puedes filtrar por los usuarios que hayan calificado la película 1 con rating mayor o igual a 4
user_map_con_valor.filter(col("rating_pelicula_1") >= 4).select("userId", "rating_pelicula_1").show()